# Імпортування даних про час кожного кола у сесії в таблиці фактів

На жаль, інформації про час кожного кола для кожного гонщика в сесії не вдалося знайти у датасетах. Проте такі дані надає бібліотека fastf1. Скористаємося її даними та імпортуємо їх у сховище, щоб мати швидкий доступ до них.

Імпортую необхідні бібліотеки

In [1]:
import fastf1
import pandas as pd
from dotenv import dotenv_values
from sqlalchemy import create_engine, text

Завантажуємо змінні оточення з .env файлу для з'єднання зі сховищем

In [2]:
config = dotenv_values()

DB_NAME = config.get('DB_NAME')
DB_USER = config.get('DB_USER')
DB_HOST = config.get('DB_HOST')
DB_PASSWORD = config.get('DB_PASSWORD')
DB_PORT = config.get('DB_PORT')

З'єднуємося з базою даних

In [3]:
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

Створюємо функцію, яка збирає дані про перегони зі сховища від обраного року до нашого часу

In [22]:
def get_races(start_year):
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT * FROM races WHERE EXTRACT(YEAR FROM date) >= {start_year} AND date <= NOW()"))
        data = result.fetchall()
        columns = result.keys()
        return pd.DataFrame(data, columns=columns)

Зберігаємо інформацію про перегони, для яких імпортуємо дані про таймінг кіл з бібліотеки в датафрейм. Дані з бібліотеки датуються починаючи з сезону 2018 і до нашого часу, тож збиратимемо дані в період з 2018-2024.

In [66]:
races_df = get_races(2018)
races_df.head()

,id,season_id,round,date,grand_prix_id,official_name,circuit_id,course_length,laps,distance
0,977,69,1,2018-03-25,4,Formula 1 2018 Rolex Australian Grand Prix,41,5.303,58,307.574
1,978,69,2,2018-04-08,7,Formula 1 2018 Gulf Air Bahrain Grand Prix,8,5.412,57,308.238
2,979,69,3,2018-04-15,12,Formula 1 2018 Heineken Chinese Grand Prix,65,5.451,56,305.066
3,980,69,4,2018-04-29,6,Formula 1 2018 Rolex Azerbaijan Grand Prix,9,6.003,51,306.049
4,981,69,5,2018-05-13,45,Formula 1 Gran Premio de España Emirates 2018,16,4.655,66,307.104


Імпортуємо потрібні нам дані з бібліотеки у відповідні датафрейми

<li>Функція <code>is_sprint_weekend</code>: визначає чи етап чемпіонату містить спринт, на вхід приймається об'єкт Event з бібліотеки fastf1, а повертається значення True або False.</li>
<li>Функція <code>fetch_data_from_session</code>: завантажує дані про конкретну сесію етапу чемпіонату, на вхід приймаються дані про рік та порядковий номер етапу чемпіонату, а також тип сесії, повертається датафрейм з відповідними даними про гонщика, час проходження кола, номер кола та команду.</li>
<li>Функція <code>get_lap_times</code>: завантажує інформацію про таймінги кіл з усіх етапів, які приймаються на вхід функції.</li>

In [73]:
def is_sprint_weekend(event):
    return event.Session3 == 'Sprint' or event.Session4 == 'Sprint'

def fetch_data_from_session(year, round, type):
    session = fastf1.get_session(year, round, type)
    session.load(
        laps = True,
        telemetry = False,
        weather = False,
        messages = False
    )

    return session.laps[['Driver', 'LapTime', 'LapNumber', 'Team']]

def get_lap_times(races_df):
    sprint_lap_times_df = pd.DataFrame()
    race_lap_times_df = pd.DataFrame()

    for index, row in races_df.iterrows():
        date = row['date']
        round = row['round']
        year = date.year
        race_id = row['id']

        event = fastf1.get_event(year=year, gp=round)
        
        if (is_sprint_weekend(event)):
            sprint_lap_times = fetch_data_from_session(year, round, 'S')
            sprint_lap_times['race_id'] = race_id
            sprint_lap_times_df = pd.concat([sprint_lap_times_df, sprint_lap_times], ignore_index=True)
        
        race_lap_times = fetch_data_from_session(year, round, 'R')
        race_lap_times['race_id'] = race_id
        race_lap_times_df = pd.concat([race_lap_times_df, race_lap_times], ignore_index=True)

    return sprint_lap_times_df, race_lap_times_df            

In [74]:
sprint_lap_times_df, race_lap_times_df = get_lap_times(races_df)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.3.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2018/1/results.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\zagrebelnio\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests_cache\session.py", line 285, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\zagrebelnio\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests_cache\session.py", line 253, in _send_and_cache
    response = super().send(request, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\zagrebelnio\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastf1\req.py", line 125, in send
    lim.limit()
  File "c:\Us

In [75]:
sprint_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id
0,HAM,NaT,1.0,Mercedes,1045
1,HAM,0 days 00:01:30.609000,2.0,Mercedes,1045
2,HAM,0 days 00:01:30.594000,3.0,Mercedes,1045
3,HAM,0 days 00:01:30.691000,4.0,Mercedes,1045
4,HAM,0 days 00:01:30.321000,5.0,Mercedes,1045


In [76]:
race_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id
0,GAS,0 days 00:01:45.060000,1.0,Toro Rosso,977
1,GAS,0 days 00:01:33.372000,2.0,Toro Rosso,977
2,GAS,0 days 00:01:32.861000,3.0,Toro Rosso,977
3,GAS,0 days 00:01:32.184000,4.0,Toro Rosso,977
4,GAS,0 days 00:01:32.332000,5.0,Toro Rosso,977


In [77]:
race_lap_times_df.info()

<class 'fastf1.core.Laps'>
RangeIndex: 143438 entries, 0 to 143437
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   Driver     143438 non-null  object         
 1   LapTime    140188 non-null  timedelta64[ns]
 2   LapNumber  143438 non-null  float64        
 3   Team       143438 non-null  object         
 4   race_id    143438 non-null  int64          
dtypes: float64(1), int64(1), object(2), timedelta64[ns](1)
memory usage: 5.5+ MB


In [78]:
sprint_lap_times_df.info()

<class 'fastf1.core.Laps'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   Driver     5365 non-null   object         
 1   LapTime    4932 non-null   timedelta64[ns]
 2   LapNumber  5365 non-null   float64        
 3   Team       5365 non-null   object         
 4   race_id    5365 non-null   int64          
dtypes: float64(1), int64(1), object(2), timedelta64[ns](1)
memory usage: 209.7+ KB


Бачимо що деякі кола не мають дані про таймінг. Це може бути пов'язано з різними причинами. Наприклад, бібліотека не враховує кола заїзду на та виїзду з піт лейну, що є логічним, оскільки ці кола не є репрезентативними відносно гоночного темпу у сесії. Також перегони бувають у режимі віртуального або реального автомобіля безпеки, який сповільнює їх темп під час аварійних інцидентів. Такі кола також можуть не враховуватися.

Тому рядки з датафрейму, що не мають таймінгу кола просто видалимо з датафреймів.

In [82]:
race_lap_times_df = race_lap_times_df.dropna(subset=['LapTime'])
race_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 140188 entries, 0 to 143437
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   Driver     140188 non-null  object         
 1   LapTime    140188 non-null  timedelta64[ns]
 2   LapNumber  140188 non-null  float64        
 3   Team       140188 non-null  object         
 4   race_id    140188 non-null  int64          
dtypes: float64(1), int64(1), object(2), timedelta64[ns](1)
memory usage: 6.4+ MB


In [84]:
sprint_lap_times_df = sprint_lap_times_df.dropna(subset=['LapTime'])
sprint_lap_times_df.info()

<class 'fastf1.core.Laps'>
Index: 4932 entries, 1 to 5364
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   Driver     4932 non-null   object         
 1   LapTime    4932 non-null   timedelta64[ns]
 2   LapNumber  4932 non-null   float64        
 3   Team       4932 non-null   object         
 4   race_id    4932 non-null   int64          
dtypes: float64(1), int64(1), object(2), timedelta64[ns](1)
memory usage: 231.2+ KB


Для того щоб підігнати дані під сховище потрібно додати новий стовпець lap_time_millis, який буде відображати час у мілісекундах. Для цього використаємо стовпець LapTime (pandas.Timedelta).

In [87]:
race_lap_times_df['lap_time_millis'] = race_lap_times_df['LapTime'].dt.total_seconds() * 1000
race_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id,lap_time_millis
0,GAS,0 days 00:01:45.060000,1.0,Toro Rosso,977,105060.0
1,GAS,0 days 00:01:33.372000,2.0,Toro Rosso,977,93372.0
2,GAS,0 days 00:01:32.861000,3.0,Toro Rosso,977,92861.0
3,GAS,0 days 00:01:32.184000,4.0,Toro Rosso,977,92184.0
4,GAS,0 days 00:01:32.332000,5.0,Toro Rosso,977,92332.0


In [88]:
sprint_lap_times_df['lap_time_millis'] = sprint_lap_times_df['LapTime'].dt.total_seconds() * 1000
sprint_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id,lap_time_millis
1,HAM,0 days 00:01:30.609000,2.0,Mercedes,1045,90609.0
2,HAM,0 days 00:01:30.594000,3.0,Mercedes,1045,90594.0
3,HAM,0 days 00:01:30.691000,4.0,Mercedes,1045,90691.0
4,HAM,0 days 00:01:30.321000,5.0,Mercedes,1045,90321.0
5,HAM,0 days 00:01:30.451000,6.0,Mercedes,1045,90451.0


Переведемо стовпці номеру кола та мілісекунд у цілочисельний тип.

In [90]:
race_lap_times_df['LapNumber'] = race_lap_times_df['LapNumber'].astype(int)
race_lap_times_df['lap_time_millis'] = race_lap_times_df['lap_time_millis'].astype(int)

race_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id,lap_time_millis
0,GAS,0 days 00:01:45.060000,1,Toro Rosso,977,105060
1,GAS,0 days 00:01:33.372000,2,Toro Rosso,977,93372
2,GAS,0 days 00:01:32.861000,3,Toro Rosso,977,92861
3,GAS,0 days 00:01:32.184000,4,Toro Rosso,977,92184
4,GAS,0 days 00:01:32.332000,5,Toro Rosso,977,92332


In [91]:
sprint_lap_times_df['LapNumber'] = sprint_lap_times_df['LapNumber'].astype(int)
sprint_lap_times_df['lap_time_millis'] = sprint_lap_times_df['lap_time_millis'].astype(int)

sprint_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id,lap_time_millis
1,HAM,0 days 00:01:30.609000,2,Mercedes,1045,90609
2,HAM,0 days 00:01:30.594000,3,Mercedes,1045,90594
3,HAM,0 days 00:01:30.691000,4,Mercedes,1045,90691
4,HAM,0 days 00:01:30.321000,5,Mercedes,1045,90321
5,HAM,0 days 00:01:30.451000,6,Mercedes,1045,90451


Тепер перетворимо стовпець LapTime у формат strinig 00:00.000, який використовується в чемпіонаті.

In [92]:
def timedelta_to_string(td):
    minutes = td.seconds // 60
    seconds = td.seconds % 60
    milliseconds = td.microseconds // 1000
    return '{:02d}:{:02d}.{:03d}'.format(minutes, seconds, milliseconds)

In [93]:
race_lap_times_df['lap_time'] = race_lap_times_df['LapTime'].dt.total_seconds().apply(lambda x: timedelta_to_string(pd.Timedelta(seconds=x)))
race_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id,lap_time_millis,lap_time
0,GAS,0 days 00:01:45.060000,1,Toro Rosso,977,105060,01:45.060
1,GAS,0 days 00:01:33.372000,2,Toro Rosso,977,93372,01:33.372
2,GAS,0 days 00:01:32.861000,3,Toro Rosso,977,92861,01:32.861
3,GAS,0 days 00:01:32.184000,4,Toro Rosso,977,92184,01:32.184
4,GAS,0 days 00:01:32.332000,5,Toro Rosso,977,92332,01:32.332


In [94]:
sprint_lap_times_df['lap_time'] = sprint_lap_times_df['LapTime'].dt.total_seconds().apply(lambda x: timedelta_to_string(pd.Timedelta(seconds=x)))
sprint_lap_times_df.head()

,Driver,LapTime,LapNumber,Team,race_id,lap_time_millis,lap_time
1,HAM,0 days 00:01:30.609000,2,Mercedes,1045,90609,01:30.609
2,HAM,0 days 00:01:30.594000,3,Mercedes,1045,90594,01:30.594
3,HAM,0 days 00:01:30.691000,4,Mercedes,1045,90691,01:30.691
4,HAM,0 days 00:01:30.321000,5,Mercedes,1045,90321,01:30.321
5,HAM,0 days 00:01:30.451000,6,Mercedes,1045,90451,01:30.451


Тепер додамо стовпці driver_id згідно з даними зі сховища. Для того щоб ідентифікувати гонщика є поле Driver з його абревіатурою. У таблиці drivers багато гонщиків могли мати ту чи іншу абревіатуру в різні періоди, проте точно ідентифікувати гонщика допоможе інше поле зі сховища, а саме його постійний номер. Справа в тому, що в чемпіонаті не так давно ввели правило постійного номеру гонщика (з 2014). Раніше гонщики не мали постійного номера, вони змінювалися кожен сезон в залежності від їхньої минулорічної позиції в чемпіонаті.

In [95]:
def get_drivers():
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT * FROM drivers"))
        data = result.fetchall()
        columns = result.keys()
        return pd.DataFrame(data, columns=columns)

In [96]:
drivers_df = get_drivers()
drivers_df.head()

,id,name,first_name,last_name,full_name,abbrevation,permanentnumber,gender,dateofbirth,nationality_country_id
0,1,Adderly Fong,Adderly,Fong,Adderly Fong Cheun-yue,FON,NaN,MALE,1990-03-02,100
1,2,Adolf Brudes,Adolf,Brudes,Adolf Brudes von Breslau,BRU,NaN,MALE,1899-10-15,84
2,3,Adolfo Schwelm Cruz,Adolfo,Schwelm Cruz,Adolfo Julio Carlos Schwelm Cruz,SCH,NaN,MALE,1923-06-28,11
3,4,Adrián Campos,Adrián,Campos,Adrián Campos Suñer,CAM,NaN,MALE,1960-06-17,210
4,5,Adrian Sutil,Adrian,Sutil,Adrian Sutil,SUT,NaN,MALE,1983-01-11,84
